# Защита персональных данных клиентов

Нужно защитить данные клиентов страховой компании «Хоть потоп». Для этого необходимо разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. 

Данные нужно защитить, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. 

## 1. Загрузка данных

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

In [2]:
data = pd.read_csv('/datasets/insurance.csv')
#data = pd.read_csv('/Users/Mikalai/Documents/Data Science/Защита персональных данных клиентов/insurance.csv')   

In [3]:
data.shape

(5000, 5)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [6]:
data.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

Изменим тип данных в столбцах Возраст и Зарплата на целочисленный

In [7]:
data[['Возраст', 'Зарплата']] = data[['Возраст', 'Зарплата']].astype('int64')

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   Пол                5000 non-null   int64
 1   Возраст            5000 non-null   int64
 2   Зарплата           5000 non-null   int64
 3   Члены семьи        5000 non-null   int64
 4   Страховые выплаты  5000 non-null   int64
dtypes: int64(5)
memory usage: 195.4 KB


In [9]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.359400,1.194200,0.148000
std,0.500049,8.440807,9900.082063,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


##### Вывод

Данные загружены.

В таблице 5000 строк и 5 столбцов.

Пропусков нет.

В столбцах Возраст и Зарплата нулевых значений нет. В данных столбцах Возраст и Зарплата изменили тип данных на целочисленный  

В данных присутствуют:
- признаки - Пол, Возраст, Зарплата, Члены семьи.
- целевой признак - Страховые выплаты.

## 2. Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Необходимо ответить на вопрос:**

**Изменится ли качество линейной регрессии при умножении матрицы признаков на обратимую матрицу?** 

**Выясним, как связаны параметры линейной регрессии в исходной задаче и в преобразованной**

Веса в линейной регрессии расчитываются по формуле

$$
w = (X^T X)^{-1} X^T y
$$

Представим новую матрицу признаков X1 как произведение старой матрицы признаков X на обратимую матрицу P:

$$
X1=XP
$$

Подставим новое значение X1 в формулу расчета весов в линейной регрессии

$$
w1 = ((XP)^T XP)^{-1} (XP)^T y
$$

Используем формулу 

$$
(AB)^T = A^T B^T
$$

и получаем

$$
w1 = (P^T X^T XP)^{-1} P^T X^T y
$$

Перегруппируем множители в скобках, и раскроем скобки

$$
w1 = (P^T (X^T X) P)^{-1} P^T X^T y = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$

Так как P обратимая матрица, то произведение  

$$
(P^T)^{-1} P^T = E
$$

это единичная матрица E. Тогда

$$
w1 = P^{-1} (X^T X)^{-1} E X^T y = P^{-1} (X^T X)^{-1} X^T y
$$

Видим, что справа получилась формула расчета весов в линейной регрессии (формула без умножения на матрицу P)

$$
w1 = P^{-1} (X^T X)^{-1} X^T y = P^{-1} w
$$

Оконательно

$$
w1 = P^{-1} w
$$

Подставим новое значение весов 𝑤1 и новую матрицу признаков Х1 в формулу для предсказаний линейной регрессии

$$
a = Xw
$$

$$
a1 = X1 w1 = X P P^{-1} w
$$

Так как  Р обратимая матрица, то произведение 

$$
P P^{-1} = E
$$

снова единичная матрица, то получаем 

$$
a1 = X P P^{-1} w = X E w = X w = a
$$

В результате получили, что предсказания a1 для матрицы признаков Х, умноженных на обратимую матрицу P равны предсказаниям a. Следовательно, при умножении признаков на обратимую матрицу качество линейной регрессии не изменится.

**Ответ:**

**при умножении матрицы признаков на обратимую матрицу качество линейной регресии не изменится**

**Параметры линейной регрессии в исходной задаче w и в преобразованной w1 связаны следующим образом**

$$
w1 = P^{-1} w
$$

## 3. Алгоритм преобразования

**Алгоритм**

1. Сгенерировать случайную квадратную матрицу по размеру равную количеству признаков (без целевого признака)

2. Проверить эту матрицу на обратимость. Для этого взять от нее обратную матрицу 

3. Умножить наши признаки на полученную матрицу 

4. Найти метрики обученной модели на исходных признаках и на признаках, умноженных на случайную квадратную матрицу

5. Сравнить метрики между собой. Если они равны, то задача выполнена

Разделим данные на признаки features и целевой признак target

In [10]:
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']

Создадим случайную обратимую матрицу размером по количеству столбцов features

In [11]:
random_matrix = np.random.normal(size = (features.shape[1], features.shape[1]))
random_matrix

array([[-1.14586821,  1.00207651,  1.29599788,  0.16273085],
       [ 0.25439558, -0.77164673, -0.48316877, -0.30157691],
       [ 0.98099068,  0.85695287,  1.24824132,  1.12084595],
       [-0.12138431, -1.17902689, -0.74925444,  1.628515  ]])

Пооверим матрицу на обратимость

In [12]:
np.linalg.inv(random_matrix)

array([[-0.44878256,  0.19746383,  0.41840109, -0.20655743],
       [-0.67836502, -2.19073062, -0.24519725, -0.1691446 ],
       [ 0.91248278,  1.95296274,  0.54633418, -0.10554221],
       [-0.10476046, -0.67281866,  0.10502604,  0.42764308]])

Перемножим исходную на обратную матрицы. Должна получиться единичная матрица

In [13]:
random_matrix @ np.linalg.inv(random_matrix)

array([[ 1.00000000e+00,  1.47944645e-16,  1.74757445e-17,
        -1.27526383e-17],
       [-2.78293992e-16,  1.00000000e+00, -9.16632986e-17,
        -5.76879305e-17],
       [ 2.79560081e-18, -2.45817165e-16,  1.00000000e+00,
        -6.92654098e-17],
       [-9.17202761e-17, -3.19704779e-16,  2.70501076e-18,
         1.00000000e+00]])

Закодируем данные. Для этого умножим исходные признаки на случайную обратимую матрицу

In [14]:
coded_features = features.dot(random_matrix)
coded_features

,0,1,2,3
0,48666.300653,42473.048114,61893.506171,55583.385545
1,37289.226621,32527.534459,47410.195047,42579.901949
2,20608.181734,17973.632612,26199.055773,23529.019149
3,40912.410860,35716.372236,52040.017886,46736.199889
4,25609.833934,22345.865995,32566.865659,29245.797784
...,...,...,...,...
4995,35028.247554,30569.253461,44547.187800,40009.013171
4996,51412.439654,44876.915614,65390.668044,58723.702503
4997,33260.429167,29032.911461,42304.218779,37993.903082
4998,32082.481890,28002.847768,40805.909603,36650.076038


Проверим совпадают ли данные после кодирования и раскодирования с исходными

Для расшифровки данных создадим функцию декодирования, которая принимает на вход зашифрованную матрицу coded_features и возвращает исходную матрицу features

In [15]:
def decode(coded_features):
    decoded_features = round(abs(coded_features.dot(np.linalg.inv(random_matrix))))
    for i in decoded_features.iloc[:, [0, 3]]:
        decoded_features[i] = decoded_features[i].astype(int)
    decoded_features.columns = features.columns
    return decoded_features

In [16]:
decode(coded_features)

,Пол,Возраст,Зарплата,Члены семьи
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
2,0,29.0,21000.0,0
3,0,21.0,41700.0,2
4,1,28.0,26100.0,0
...,...,...,...,...
4995,0,28.0,35700.0,2
4996,0,34.0,52400.0,1
4997,0,20.0,33900.0,2
4998,1,22.0,32700.0,3


Сравним расшифрованные данные с исходными данными features

In [17]:
features

,Пол,Возраст,Зарплата,Члены семьи
0,1,41,49600,1
1,0,46,38000,1
2,0,29,21000,0
3,0,21,41700,2
4,1,28,26100,0
...,...,...,...,...
4995,0,28,35700,2
4996,0,34,52400,1
4997,0,20,33900,2
4998,1,22,32700,3


Данные после кодирования и декодирования полностью совпадают с исходными

##### Выводы 

Предложенный алгоритм преобразования данных справляется для решения задачи.

Данные после кодирования и декодирования полностью совпадают с исходными

## 4. Проверка алгоритма

Проверим, что качество линейной регрессии из sklearn не отличается до и после преобразования. Применим метрику R2.

Создадим модель линейной регрессии и обучим на 2 обучающих матрицах:
- исходной features
- кодированной coded_features

In [18]:
sklearn_model = LinearRegression().fit(features, target)
predictions = sklearn_model.predict(features)
print(f'Значение метрики R2 на исходных данных: {r2_score(target, predictions):.10f}')

Значение метрики R2 на исходных данных: 0.4249455031


In [19]:
sklearn_model.fit(coded_features, target)
predictions = sklearn_model.predict(coded_features)
print(f'Значение метрики R2 на кодированных данных: {r2_score(target, predictions):.10f}')

Значение метрики R2 на кодированных данных: 0.4249455031


##### Вывод

Значения метрики R2 на исходных данных и метрики R2 на кодированных данных совпадают.

Качество линейной регрессии не изменилось.

Алгоритм преобразования данных работает правильно.

### Итоговый вывод

Проект выполнен для страховой компании «Хоть потоп» с целью защиты персональных данных клиентов.

При выполнения проекта:

1. Загружены и изучены данные. При этои определили, что 
- данные содержат 5000 строк и 5 столбцов
- пропусков данных нет
- в данных присутствуют: признаки - Пол, Возраст, Зарплата, Члены семьи; и целевой признак - Страховые выплаты
- в столбцах Возраст и Зарплата изменили тип данных на целочисленный  
- другой дополнительной предобработки данных не потребовалось 

2. С помощью формул выяснили что
- при умножении признаков на обратимую матрицу качество линейной регрессии не изменится
- предсказания "a1" для матрицы признаков Х, умноженных на обратимую матрицу P равны предсказаниям "a"
- параметры линейной регрессии в исходной задаче w и в преобразованной w1 связаны следующей зависимостью 
$$
w1 = P^{-1} w
$$

3. Предложен и проверен алгоритм преобразования данных для решения задачи защиты персональных данных клиентов страховой компании 
- предложенный алгоритм преобразования данных справляется для решения задачи шифрования признаков
- предложенный алгоритм шифрования данных позволяет дешифровать данные при необходимости 
- применение при шифровании случайной обратимой матрицы позволяет при каждом запуске получать новые зашифрованные данные, поэтому восстановить исходные данные без кода программы будет очень сложно 

4.	Применив метрику R2 проверили, что качество линейной регрессии из sklearn не отличается до и после преобразования
- значения метрики R2 на исходных данных и метрики R2 на кодированных данных совпадают
- качество линейной регрессии не изменилось
- алгоритм преобразования данных работает правильно

**Таким образом, предложенные и реализованный алгоритм полностью удовлетворяет поставленной задаче по защите персональных данных клиентов страховой компании**